In [1]:
from djitellopy import Tello

# Drone Flying Script

In [77]:
from djitellopy import Tello
import time

def flyer(gesture):
    
    drone = Tello()

    
    print("Connecting to the drone...")
    drone.connect()
    print(f"Battery level: {drone.get_battery()}%")
    
    
    print("Taking off...")
    drone.takeoff()
    time.sleep(2)  
    

    try:
        
        if gesture == 'up':
            print("Moving up...")
            drone.move_up(100)  
        elif gesture == 'down':
            print("Moving down...")
            drone.move_down(70)  
        elif gesture == 'left':
            print("Moving left...")
            drone.move_left(100)  
        elif gesture == 'right':
            print("Moving right...")
            drone.move_right(100)  
        elif gesture == 'splay':
            print("Performing a backflip...")
            drone.flip_back()  
        elif gesture == 'finger_down_right':
            print("Performing a right backflip...")
            drone.flip_right()  
        elif gesture == 'finger_down_left':
            print("Performing a left backflip...")
            drone.flip_left()  
        else:
            print("Hovering instead...")
    except Exception as e:
        print(f"Error performing movement: {e}")

    
    print("Hovering...")
    time.sleep(2)

    
    print("Landing...")
    drone.land()
    print("Landed successfully.")



# Online Gesture Classification ML Pipeline

In [ ]:
import serial
import time
import numpy as np
from collections import deque
import joblib  


# model = joblib.load('models\Models\random_forest_model.pkl')  


# ser = serial.Serial('COM7', 115200)  
# time.sleep(2)  


# WINDOW_SIZE = 200  
# BUFFER_SIZE = 400  


# buffer = deque(maxlen=BUFFER_SIZE)


# def process_line(line):

#     try:
#         if "mpu6050 test:" in line:
            
#             if "MPU1" in line:
#                 sensor_name = "MPU1"
#             elif "MPU2" in line:
#                 sensor_name = "MPU2"
#             else:
#                 return None  

            
#             parts = line.split(',')
#             parts = [item.replace('\x1b[0;32m', '').replace('\x1b[0m', '').strip() for item in parts]

            
#             feature_vector = []
#             for part in parts:
#                 if ':' in part:
#                     key, value = part.split(':')
#                     feature_vector.append(float(value.strip()))
#             return feature_vector
#     except Exception as e:
#         print(f"Error processing line: {line}, Error: {e}")
#         return None


# def classify_gestures():
#     print("Starting real-time gesture classification...")
#     window = deque(maxlen=WINDOW_SIZE)  

#     while True:
#         try:
#             if ser.in_waiting > 0:  
#                 line = ser.readline().decode('utf-8', errors='replace').strip()
#                 feature_vector = process_line(line)

#                 if feature_vector:
                    
#                     buffer.append(feature_vector)

                   
#                     window.append(feature_vector)

                    
#                     if len(window) == WINDOW_SIZE:
                        
#                         input_data = np.array(window)

                      
#                         input_data_flat = input_data.flatten().reshape(1, -1)

                        
#                         prediction = model.predict(input_data_flat)
#                         print(f"Prediction: {prediction[0]}")  

#         except KeyboardInterrupt:
#             print("Real-time gesture classification stopped.")
#             break
#         except Exception as e:
#             print(f"Error: {e}")
#             continue


# classify_gestures()


In [49]:
import re
import os
import serial
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from collections import deque
from scipy.fft import fft
from collections import Counter

# Patterns to match MPU1 and MPU2 data
pattern = re.compile(r'(MPU\d) acce_x:([-+]?\d*\.\d+|\d+), acce_y:([-+]?\d*\.\d+|\d+), acce_z:([-+]?\d*\.\d+|\d+), gyro_x:([-+]?\d*\.\d+|\d+), gyro_y:([-+]?\d*\.\d+|\d+), gyro_z:([-+]?\d*\.\d+|\d+)')
# acce_pattern = re.compile(r'(MPU\d) ')

def parse_data(line):
    match = pattern.search(line)
    if match:
        sensor, acce_x, acce_y, acce_z, gyro_x, gyro_y, gyro_z = match.groups()
        return gyro_x, gyro_y, gyro_z, acce_x, acce_y, acce_z, sensor
    return None

"""
MPU1 2D List: [[gyro_x, gyro_y, gyro_z, acce_x, acce_y, acce_z, sensor], ...]
MPU2 2D List: [[gyro_x, gyro_y, gyro_z, acce_x, acce_y, acce_z, sensor], ...]

Concat sensor data from MPU1 and MPU2 into a single DataFrame
then get the first 10 features of fft for each column
"""
def ml_parse_data(mpu1, mpu2):
    # Create a DataFrame from the MPU1 and MPU2 data and concatenate them without sensor column
    assert len(mpu1) == len(mpu2)
    assert len(mpu1) > 0 and len(mpu2) > 0
    assert len(mpu1[0]) == 7 and len(mpu2[0]) == 7
    mpu1_df = pd.DataFrame(mpu1, columns=['gyro_x_1', 'gyro_y_1', 'gyro_z_1', 'acce_x_1', 'acce_y_1', 'acce_z_1', 'sensor_1'])
    mpu2_df = pd.DataFrame(mpu2, columns=['gyro_x_2', 'gyro_y_2', 'gyro_z_2', 'acce_x_2', 'acce_y_2', 'acce_z_2', 'sensor_2'])
    mpu1_df = mpu1_df.drop(columns=['sensor_1']).reset_index(drop=True)
    mpu2_df = mpu2_df.drop(columns=['sensor_2']).reset_index(drop=True)
    df = pd.concat([mpu1_df, mpu2_df], axis=1)

    fft_features = []
    for col in df.columns:

        fft_values = fft(df[col].values)

        fft_mag = np.abs(fft_values)[:10]

        fft_features.extend(fft_mag)

    return fft_features

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

import pickle

def predict_gesture(X):
    
    with open(r'models\Models\random_forest_model.pkl', 'rb') as file:
        model = pickle.load(file)
    
    y_pred = model.predict(X)
    return y_pred

def voter(gest_list):
    gesture_counts = Counter(gest_list)  
    most_common_gesture = gesture_counts.most_common(1)  
    
    return most_common_gesture[0][0]

In [181]:
def read_data():
    try:
        
        gest_list = []
        
        with serial.Serial('COM8', 115200, timeout=1) as ser:
            print(f"Connected to {'COM8'} at {115200} baud.")
            print("Reading IMU data... Press stop to interrupt.")
            
            mpu1_buffer = deque(maxlen=100)
            mpu2_buffer = deque(maxlen=100)

            while True:
                
                line = ser.readline().decode('utf-8').strip()

                data = parse_data(line)

                if data is not None:
                    gx, gy, gz, ax, ay, az, mpu_id = data
                
                    if mpu_id == 'MPU1':
                        mpu1_buffer.append(data)
                    elif mpu_id == 'MPU2':
                        mpu2_buffer.append(data)
                    
                    if len(mpu1_buffer) == 100 and len(mpu2_buffer) == 100:

                        map_gesture = {
                            0:'down',
                            1:'finger_down_left',
                            2:'finger_down_right',
                            3:'idle',
                            4:'left',
                            5:'right',
                            6:'splay',
                            7:'up',
                        }
    
                        freak = ml_parse_data(mpu1_buffer, mpu2_buffer)

                        fft_features_mpu = np.array(freak).reshape(1,-1)

                        gesture = predict_gesture(fft_features_mpu)
                        gest_list.append(map_gesture[gesture[0]])
        
                        if len(gest_list) == 30:
                            print(voter(gest_list))
                            flyer(voter(gest_list))
                            break
                        
        
        
                        
                        
                
    except KeyboardInterrupt:
        print("\nStopped by user.")
    except serial.SerialException as e:
        print(f"Error: {e}")

In [186]:
read_data()

Connected to COM8 at 115200 baud.
Reading IMU data... Press stop to interrupt.


[INFO] tello.py - 129 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 438 - Send command: 'command'
[ERROR] tello.py - 458 - 'utf-8' codec can't decode byte 0xcc in position 0: invalid continuation byte
[INFO] tello.py - 438 - Send command: 'command'
[INFO] tello.py - 462 - Response command: 'ok'


idle
Connecting to the drone...
Battery level: 87%
Taking off...


[INFO] tello.py - 438 - Send command: 'takeoff'
[INFO] tello.py - 462 - Response takeoff: 'ok'


Hovering instead...
Hovering...


[INFO] tello.py - 438 - Send command: 'land'


Landing...


[WARNING] tello.py - 448 - Aborting command 'land'. Did not receive a response after 7 seconds
[INFO] tello.py - 438 - Send command: 'land'
[INFO] tello.py - 462 - Response land: 'ok'


Landed successfully.
